<a href="https://colab.research.google.com/github/ChoiDae1/Youtube_Project/blob/main/%EC%9C%A0%ED%8A%9C%EB%B8%8C_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [12]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [13]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import sys
import requests
import pandas as pd

In [14]:
it_과학기술 = {
    'ZHC': 'https://www.youtube.com/c/ZHcomicart/videos',
    'Mark Rober': 'https://www.youtube.com/c/MarkRober/videos',
    'Apple': 'https://www.youtube.com/user/Apple/videos',
    'Marques Brownlee': 'https://www.youtube.com/c/mkbhd/videos',
    'Veritasium': 'https://www.youtube.com/c/veritasium/videos',
    'CrazyRussianHacker': 'https://www.youtube.com/user/CrazyRussianHacker/videos',
    'SmarterEveryDay': 'https://www.youtube.com/c/smartereveryday/videos',
    'WIRED': 'https://www.youtube.com/c/WIRED/videos',
    'EverythingApplePro E A P': 'https://www.youtube.com/c/EverythingApplePro/videos',
    '#Mind Warehouse': 'https://www.youtube.com/channel/UCYenDLnIHsoqQ6smwKXQ7Hg/videos',
    'JerryRigEverything': 'https://www.youtube.com/c/JerryRigEverything/videos',
    'iJustine': 'https://www.youtube.com/c/iJustine/videos',
    'hickok45': 'https://www.youtube.com/user/hickok45/videos',
    'freeCodeCamp.org': 'https://www.youtube.com/c/Freecodecamp/videos',
    'Austin Evans': 'https://www.youtube.com/c/austinevans/videos',
    'TheBackyardScientist': 'https://www.youtube.com/c/TheBackyardScientist/videos',
    'Seeker': 'https://www.youtube.com/c/Seeker/videos',
    'Ridddle': 'https://www.youtube.com/c/VineMontanaTV/videos',
    'Be Smart': 'https://www.youtube.com/c/itsokaytobesmart/videos',
    'The Organic Chemistry Tutor': 'https://www.youtube.com/c/TheOrganicChemistryTutor/videos'
              }
          

In [15]:
영화_애니 = {
    'Movieclips': 'https://www.youtube.com/c/MOVIECLIPS/videos',
    'TheEllenShow': 'https://www.youtube.com/c/TheEllenShow/videos',
    'Masha and The Bear': 'https://www.youtube.com/c/MashaBearEN/videos',
    'Netflix': 'https://www.youtube.com/c/Netflix/videos',
    'WB Kids': 'https://www.youtube.com/c/wbkids/videos',
    'REACT': 'https://www.youtube.com/finebros/videos',
    'Pencilmation': 'https://www.youtube.com/pencilmation/videos',
    'Alan Becker': 'https://www.youtube.com/c/noogai89/videos',
    'Marvel Entertainment': 'https://www.youtube.com/c/marvel/videos',
    'PDK Films': 'https://www.youtube.com/c/PDKFilms/videos',
    'Movieclips Trailers': 'https://www.youtube.com/c/MovieclipsTRAILERS/videos',
    'LEGO': 'https://www.youtube.com/c/LEGO/videos',
    'Annoying Orange': 'https://www.youtube.com/c/AnnoyingOrange/videos',
    'Chotoonz TV - Funny Cartoons for Kids': 'https://www.youtube.com/c/ChotoonzTV/videos',
    'Jaiden Animations': 'https://www.youtube.com/c/jaiden/videos',
    'WiederDude': 'https://www.youtube.com/channel/UCpDNnvxEf3iWuJbtmUk6DnA/videos',
    'Comedy Central': 'https://www.youtube.com/c/ComedyCentral/videos',
    'The Film Theorists': 'https://www.youtube.com/c/FilmTheorists/videos',
    'Warner Bros. Pictures': 'https://www.youtube.com/wbpictures/videos',
} # 마지막 하나는 홈페이지 자체에 접근할 수 없어서 제외

In [16]:
자동차 = {
    'Its Mamix': 'https://www.youtube.com/c/ItsMamix/videos',
    'The Q': 'https://www.youtube.com/c/TheQ_original/videos',
    'Tanner Fox': 'https://www.youtube.com/c/TannerFox/videos',
    'Life Hacks & Experiments': 'https://www.youtube.com/user/SerbianGamesBL2/videos',
    'ChrisFix': 'https://www.youtube.com/c/chrisfix/videos',
    'DenLion TV': 'https://www.youtube.com/c/DenLionTV/videos',
    'MotorTrend Channel': 'https://www.youtube.com/c/MotorTrendWatch/videos',
    'Donut Media': 'https://www.youtube.com/c/DonutMediaTV/videos',
    'Creative Channel': 'https://www.youtube.com/c/ChannelCreativeVN/videos',
    'Scotty Kilmer': 'https://www.youtube.com/c/ScottyKilmermechanic/videos',
    'Hoonigan': 'https://www.youtube.com/c/TheHoonigans/videos',
    'Salomundo': 'https://www.youtube.com/c/Salomundo/videos',
    'Edwin Sarkissian': 'https://www.youtube.com/c/EdwinSarkissian/videos',
    'TheStradman': 'https://www.youtube.com/c/TheStradman/videos',
    'KuDo Review': 'https://www.youtube.com/c/KuDoToysReview/videos',
    'Adam LZ': 'https://www.youtube.com/c/AdamLZ/videos',
    'Jay Lenos Garage': 'https://www.youtube.com/c/jaylenosgarage/videos',
    '1320video': 'https://www.youtube.com/user/1320video/videos',
    'Engineering Explained': 'https://www.youtube.com/user/EngineeringExplained/videos',
    'RE4LG4LIFE OFICIAL': 'https://www.youtube.com/c/RE4LG4LIFEOFICIAL/videos'
}

In [21]:
# 유튜버 한명 url(/videos로 끝남) 하나 받으면 최신순으로 영상 100개에 대한 크롤링 반환하는 함수 
def crawling_function(url):
    # 웹 드라이버 초기화
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.implicitly_wait(5) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    image_list = list() # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = list() # 썸네일 제목을 저장하는 리스트
    view_list = list() # 조회수를 저장하는 리스트
    url_list = list() # url을 저장하는 리스트
    intro_list = list()

    idx = 1
    while True:

        try:
            img_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'
            viewcnt_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/span[1]'
            
            # 이미지가 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            img = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, img_xpath)))
            if img is None:
                print(idx, 'img is not loaded.')
            
            # 한 페이지에 약 8개 불러오는 데, 동영상 목록을 추가 불러오기 위해 스크롤 내림
            if idx % 8 == 0 :
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
                driver.execute_script('window.scrollBy(0, 1080);')
                time.sleep(2)
            
            # 썸네일 주소
            image = driver.find_element_by_xpath(img_xpath)
            img_url = image.get_attribute('src')
  
            # 타이틀
            title = driver.find_element_by_xpath(title_xpath)
            title = title.text

            # 조회수
            view = driver.find_element_by_xpath(viewcnt_xpath)
            view = view.text
            
            # 한꺼번에 저장(셋 중에 하나라도 오류날 수 있으므로)
            image_list.append(img_url)
            title_list.append(title)
            view_list.append(view)

            if idx == 100:
              break
            
            idx += 1

        except Exception as e:
            print()
            print(e)
            break
    print("finish")
    #링크 크롤링
    page = driver.page_source
    soup = bs(page,'lxml')
    links = soup.find_all("div",id="dismissible")
    for link in links:
        url_list.append('https://www.youtube.com'+link.a['href'])
        if len(url_list) == 100:
          break

    # 설명 크롤링
    for link in url_list:
        driver.get(link)        
        time.sleep(5)
        page = driver.page_source
        soup = bs(page,'lxml')
        find = soup.find("span",class_="style-scope yt-formatted-string") 
        # find 객체가 None인 경우도 있기 때문에 이런 경우는 무시-> 설명을 None으로 대체
        if find == None:
            intro_list.append("None")
            continue

        spann = find.get_text()
        intro_list.append(spann)
    
    assert len(image_list) == len(title_list) == len(view_list) == len(url_list) == len(intro_list)
    driver.close()
    print('finish!')
    return image_list, title_list, view_list, url_list, intro_list

In [ ]:
image_list, title_list, view_list, url_list, intro_list = crawling_function('https://www.youtube.com/c/WIRED/videos')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


In [ ]:
full_image_list1 = list()
full_title_list1 = list()
full_view_list1 = list()
full_url_list1 = list()
full_intro_list1 = list()

for youtuber, url in it_과학기술.items():
  image_list, title_list, view_list, url_list, intro_list = crawling_function(url)
  full_image_list1 += image_list
  full_title_list1 += title_list
  full_view_list1 += view_list
  full_url_list1 += url_list
  full_intro_list1 += intro_list

In [ ]:
full_image_list2 = list()
full_title_list2 = list()
full_view_list2 = list()
full_url_list2 = list()
full_intro_list2 = list()

for youtuber, url in 영화_애니.items():
  image_list, title_list, view_list, url_list, intro_list = crawling_function(url)
  full_image_list2 += image_list
  full_title_list2 += title_list
  full_view_list2 += view_list
  full_url_list2 += url_list
  full_intro_list2 += intro_list

In [ ]:
full_image_list3 = list()
full_title_list3 = list()
full_view_list3 = list()
full_url_list3 = list()
full_intro_list3 = list()

for youtuber, url in 자동차.items():
  image_list, title_list, view_list, url_list, intro_list = crawling_function(url)
  full_image_list3 += image_list
  full_title_list3 += title_list
  full_view_list3 += view_list
  full_url_list3 += url_list
  full_intro_list3 += intro_list

In [ ]:
last_image_list = full_image_list1 + full_image_list2 + full_image_list3
last_title_list = full_title_list1 + full_title_list2 + full_title_list3
last_view_list = full_view_list1 + full_view_list2 + full_view_list3
last_url_list = full_url_list1 + full_url_list2 + full_url_list3
last_into_list = full_intro_list1 + full_intro_list2 + full_intro_list3

In [ ]:
import pandas as pd

final_crawling = pd.DaraFrame({
    'image': last_image_list,
    'title': last_title_list,
    'view': last_view_list,
    'url': last_url_list,
    'intro': last_into_list
})
final_crawling.to_csv('/content/drive/MyDrive/유튜브 썸네일 크롤링/crawling_data.csv')